# Insper - Ciência dos Dados 2022.2

## Engenharia - 2ºB

### Integrantes do grupo:

* Beatriz Rodrigues de Freitas

* Carlos Eduardo Porciuncula Yamada

* Verônica Luisa Ribeiro Lima

___
<div id='sumario'></div>

## Sumário

1. [Introdução](#intro)

2. [Minerando Dados e Características do Dataset](#data-mining)

3. [Modelos de Predição](#prediction)

4. [Processo e Estatísticas de Validação](#validation)

5. [Conclusão](#conclusion)

6. [Referências Bibliográficas](#ref)

___
<div id='intro'></div>

## 1. Introdução

### Objetivo

(Objetivo escolhido para trabalhar neste projeto)

### Descrição da base de dados

* `age`: idade do paciente (anos);

* `anaemia`: **booleano** → 0, se o paciente não tem anemia; 1, se o paciente tem anemia;

* `high blood pressure`: **booleano** → 0, se o paciente não tem hipertensão; 1, se o paciente tem hipertensão;

* `creatinine phosphokinase` (*CPK*): concentração da enzima CPK no sangue (mcg/L);

* `diabetes`: **booleano** → 0, se o paciente não possui; 1, se o paciente possui;

* `ejection fraction`: porcentagem do sangue que sai do coração a cada contração (porcentagem, %);

* `platelets`: plaquetas no sangue (kiloplaquetas/mL);

* `sex`: **booleano** → 0, se mulher; 1, se homem;

* `serum creatinine`: concentração de creatinina sérica no sangue (mg/dL);

* `serum sodium`: concentração de sódio sérico no sangue (mEq/L);

* `smoking`: **booleano** → 0, se o paciente não fuma; 1, se o paciente fuma;

* `time`: tempo de acompanhamento (dias);

* `death event` (*target*): **booleano** → 0, o evento não ocorre; 1, o evento não ocorre;



In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
import os
import re 
import matplotlib.pyplot as plt


In [2]:
train = pd.read_excel(f'\dados.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\dados.xlsx'

[Voltar para o sumário](#sumario)

___
<div id='data-mining'></div>

## 2. Minerando Dados e Características do Dataset

* Se necessário, faça filtro na base de dados tanto de linhas como de colunas em prol do
objetivo traçado anteriormente.

* Descreva as variáveis finais.

* Faça análise descritiva detalhada das variáveis, norteado pelo objetivo do problema. Aqui, é
interessante entender como sua variável *target* (y) se comporta cruzada com cada *feature* (x). Ferramentas estatísticas: medidas de associação, gráfico de dispersão, entre outras.

* *Storytelling* com dados: explicação do raciocínio.

* Link do <a href='https://vimeo.com/114900089'>vídeo</a> sobre Teoria das Cores para ajudar no *storytelling*.



[Voltar para o sumário](#sumario)

___
<div id='prediction'></div>

## 3. Modelos de Predição

Descrever e justificar pelo menos **DUAS** técnicas de predição.

[Voltar para o sumário](#sumario)

___
<div id='validation'></div>

## 4. Processo e Estatísticas de validação

Calcular medidas que informam a performance de cada modelo ajustado. Para cada um:

* Dividir a base de dados em **Treinamento** e **Teste**. Usar **Treinamento** para estimar cada modelo.

* Estudar medidas que validam os modelos.

    Exemplos:

    * <a href='https://help.salesforce.com/s/articleView?id=sf.bi_edd_model_metrics_cross_validation_measure.htm&type=5'>Cross Validation Tab for Numeric Use Cases</a>

    * <a href='https://paulovasconcellos.com.br/como-saber-se-seu-modelo-de-machine-learning-está-funcionando-mesmo-a5892f6468b'>Como saber se seu modelo de Machine Learning está funcionando mesmo</a>

    * <a href='https://ensinandomaquinasblog.wordpress.com/2017/12/15/modelos-preditivos-de-notas-de-redacao-do-enem-2015/'>Modelos Preditivos de Notas de Redação do ENEM 2015</a>

* Discutir sobre overfitting e underfitting (<a href='https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/'>link</a>)

* **Extra**: Faça o processo de Validação Cruzada utilizando também 10 ciclos e calcule a performance média e desvio padrão das duas medidas R2 e RMS tanto para a parte treinamento como para a parte teste. Discuta com riqueza de detalhes.

[Voltar para o sumário](#sumario)

___
<div id='conclusion'></div>

## 5. Conclusão

[Voltar para o sumário](#sumario)

___
<div id='ref'></div>

## 6. Referências Bibliográficas

[Voltar para o sumário](#sumario)